In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import mnist_inference
import os
mnist = input_data.read_data_sets("./MNIST DATA", one_hot=True)

C:\Users\Philip\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./MNIST DATA\train-images-idx3-ubyte.gz
Extracting ./MNIST DATA\train-labels-idx1-ubyte.gz
Extracting ./MNIST DATA\t10k-images-idx3-ubyte.gz
Extracting ./MNIST DATA\t10k-labels-idx1-ubyte.gz


In [3]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAING_STEP = 6000
MOVING_AVERAGE_DECAY = 0.99

MODEL_SAVE_PATH = "./save"
MODEL_NAME = "model.ckpt"

In [4]:
def train(mnist):
    x = tf.placeholder(tf.float32, shape=[None, mnist_inference.INPUT_NODE], name="X-input")
    y = tf.placeholder(tf.float32, shape=[None, mnist_inference.OUTPUT_NODE], name="Y-input")
    
    regulatizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
    y_ = mnist_inference.inference(x, regulatizer)
    
    #定義存儲訓練輪數的變量
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)   
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_ , 
                                                                   labels=tf.argmax(y, 1))
    #計算當前batch中所有cross_entropy平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, 
                                               global_step, 
                                               mnist.train.num_examples / BATCH_SIZE, 
                                               LEARNING_RATE_DECAY)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #先不執行optimizer先執行variable_averages_op
    with tf.control_dependencies([optimizer, variable_averages_op]):
        train_op = tf.no_op(name="train")
    
    #初始化持久化類
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAING_STEP):
                              
            xs, ys = mnist.train.next_batch(BATCH_SIZE)

            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x : xs, y:ys})
            
            if (i % 1000 == 0):
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

In [5]:
tf.reset_default_graph()
train(mnist)

InternalError: Blas GEMM launch failed : a.shape=(100, 784), b.shape=(784, 500), m=100, n=500, k=784
	 [[Node: layer1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_X-input_0_0/_17, layer1/weights/read)]]
	 [[Node: GradientDescent/update/_26 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_207_GradientDescent/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'layer1/MatMul', defined at:
  File "C:\Users\Philip\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Philip\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_ins